# CREATING A FACT TABLE

**Reading the silver table**

In [0]:
df_silver = spark.sql("select * from parquet.`abfss://silver@storageadlsgen3.dfs.core.windows.net/carsales`")
display(df_silver)

In [0]:
df_model = spark.table("project01.gold.dim_model")
df_branch = spark.table("project01.gold.dim_branch")
df_dealer = spark.table("project01.gold.dim_dealer")
df_date = spark.table("project01.gold.dim_date")

In [0]:
df_fact = df_silver.join(df_model, df_silver['Model_ID'] == df_model['Model_ID'])\
                     .join(df_branch, df_silver['Branch_ID'] == df_branch['Branch_ID'])\
                     .join(df_dealer, df_silver['Dealer_ID'] == df_dealer['Dealer_ID'])\
                     .join(df_date, df_silver['Date_ID'] == df_date['Date_ID'])\
                     .select(df_silver['Revenue'], df_silver['Units_Sold'], df_silver['price'], df_model['dim_model_key'], df_branch['dim_branch_key'], df_dealer['dim_dealer_key'], df_date['dim_date_key'])

In [0]:
display(df_fact)

**Merge statement using sql**

In [0]:
if spark.catalog.tableExists("project01.gold.factsales"):
    df_fact.createOrReplaceTempView("factsales_view")
    merge_statement = """ MERGE INTO project01.gold.factsales as t
                          USING factsales_view as s
                          ON t.dim_model_key = s.dim_model_key AND t.dim_branch_key = s.dim_branch_key AND t.dim_dealer_key = s.dim_dealer_key
                          WHEN MATCHED THEN UPDATE SET *
                          WHEN NOT MATCHED THEN INSERT *

    """
    spark.sql(merge_statement)
else:
    df_fact.write\
           .format("delta")\
           .mode("overwrite")\
           .option("path","abfss://gold@storageadlsgen3.dfs.core.windows.net/factsales")\
           .saveAsTable("project01.gold.factsales")

In [0]:
%sql
select * from project01.gold.factsales

**Merge statement using Python**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("project01.gold.factsales"):
    delta_tb = DeltaTable.forPath(spark, "abfss://gold@storageadlsgen3.dfs.core.windows.net/factsales")
    delta_tb.alias("t").merge(df_fact.alias("s"), "t.dim_model_key = s.dim_model_key AND t.dim_branch_key = s.dim_branch_key AND t.dim_dealer_key = s.dim_dealer_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_fact.write\
           .format("delta")\
           .mode("overwrite")\
           .option("path", "abfss://gold@storageadlsgen3.dfs.core.windows.net/factsales")\
           .option("mergeSchema", "true")\
           .saveAsTable("project01.gold.factsales")

In [0]:
%sql
select * from project01.gold.factsales